<a href="https://colab.research.google.com/github/arminak6/Control-Quality/blob/Dev/Code/anomalib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anomalib

In [2]:
!pip install lightning
!pip install kornia
!pip install FrEIA
!pip install python-dotenv
!pip install open-clip-torch
!pip install openvino
!pip install openvino-dev
!pip install onnx
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for FrEIA: filename=FrEIA-0.2-py3-none-any.whl size=42762 sha256=d1e6a3056175c1788e13f6b616c6495b065a183524d2cf497ee35b0f8c46d961
  Stored in directory: /root/.cache/pip/wheels/81/a8/e2/d532a76f72108ac4a340cbe3f86b4f591abfdbd75209a5badb
Successfully built FrEIA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [5]:
import os
import torch
from torchvision.transforms.functional import to_pil_image
from anomalib.data import Folder
from anomalib.models import EfficientAd
from anomalib import TaskType
from anomalib.engine import Engine
from anomalib.data.utils import ValSplitMode



# Dataset Configuration
datamodule = Folder(
    name="custom_dataset",  # Add dataset name here
    root="/content/drive/MyDrive/praeciso/anomalib_dataset",
    normal_dir="train/good",
    abnormal_dir="test/anomalous",
    extensions=[".bmp"],
    task= TaskType.CLASSIFICATION,
    seed=42,
    normal_split_ratio=0.2, # default value
    val_split_mode=ValSplitMode.FROM_TEST, # default value
    val_split_ratio=0.5, # default value
    train_batch_size=32, # default value
    eval_batch_size=32, # default value
)

# Set batch sizes
datamodule.train_batch_size = 1
datamodule.eval_batch_size = 1
datamodule.setup()




In [6]:
# Train images
i, data_train = next(enumerate(datamodule.train_dataloader()))
print(data_train.keys(), data_train["image"].shape) # it takes a batch of images
# for each key extract the first image
print("data_train['image_path'][0]: {} - data_train['image'][0].shape: {} - data_train['label'][0]: {} - torch.max(data_train['image][0]): {} - torch.min(data_train['image][0]): {}".format(data_train['image_path'][0], data_train['image'][0].shape, data_train['label'][0], torch.max(data_train['image'][0]), torch.min(data_train['image'][0])))
img_train = to_pil_image(data_train["image"][0].clone())

# val images
i, data_val = next(enumerate(datamodule.val_dataloader()))
# for each key extract the first image
print("data_val['image_path'][0]: {} - data_val['image'][0].shape: {} - data_val['label'][0]: {}".format(data_val['image_path'][0], data_val['image'][0].shape, data_val['label'][0]))
img_val = to_pil_image(data_val["image"][0].clone())

# test images
i, data_test = next(enumerate(datamodule.test_dataloader()))
# for each key extract the first image
print("data_test['image_path'][0]: {} - data_test['image'][0].shape: {} - data_test['label'][0]: {}".format(data_test['image_path'][0], data_test['image'][0].shape, data_test['label'][0]))
img_test = to_pil_image(data_test["image"][0].clone())

# from the datamodule extract the train, val and test Pandas dataset and collect all the info in a csv
train_dataset = datamodule.train_data.samples
test_dataset = datamodule.test_data.samples
val_dataset = datamodule.val_data.samples

# check the data distribution for each category in each data split
print("TRAIN DATASET FEATURES")
print(train_dataset.info())
print("")
print("IMAGE DISTRIBUTION BY CLASS")
print("")
desc_grouped = train_dataset[['label']].value_counts()
print(desc_grouped)
print("----------------------------------------------------------")
print("TEST DATASET FEATURES")
print(test_dataset.info())
print("")
print("IMAGE DISTRIBUTION BY CLASS")
print("")
desc_grouped = test_dataset[['label']].value_counts()
print(desc_grouped)
print("----------------------------------------------------------")
print("VAL DATASET FEATURES")
print(val_dataset.info())
print("")
print("IMAGE DISTRIBUTION BY CLASS")
print("")
desc_grouped = val_dataset[['label']].value_counts()
print(desc_grouped)

datamodule.train_data.samples.to_csv(os.path.join("/content/drive/MyDrive/praeciso/show", "datamodule_train.csv"), index=False)
datamodule.test_data.samples.to_csv(os.path.join("/content/drive/MyDrive/praeciso/show", "datamodule_test.csv"), index=False)
datamodule.val_data.samples.to_csv(os.path.join("/content/drive/MyDrive/praeciso/show", "datamodule_val.csv"), index=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


dict_keys(['image_path', 'label', 'image']) torch.Size([1, 3, 1536, 2048])
data_train['image_path'][0]: /content/drive/MyDrive/praeciso/anomalib_dataset/train/good/OK4_30K_97_05.bmp - data_train['image'][0].shape: torch.Size([3, 1536, 2048]) - data_train['label'][0]: 0 - torch.max(data_train['image][0]): 1.0 - torch.min(data_train['image][0]): 0.0
data_val['image_path'][0]: /content/drive/MyDrive/praeciso/anomalib_dataset/test/anomalous/KO1_30K_97_00.bmp - data_val['image'][0].shape: torch.Size([3, 1536, 2048]) - data_val['label'][0]: 1
data_test['image_path'][0]: /content/drive/MyDrive/praeciso/anomalib_dataset/test/anomalous/KO1_30K_97_02.bmp - data_test['image'][0].shape: torch.Size([3, 1536, 2048]) - data_test['label'][0]: 1
TRAIN DATASET FEATURES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_path   60 non-null     object
 1   label   

In [14]:
from anomalib.models import ReverseDistillation
from anomalib import TaskType
from anomalib.data.image.folder import Folder
from anomalib.engine import Engine
from anomalib.deploy import ExportType
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

# 1 - instantiate the model
model = EfficientAd()

# 2 - instantiate the callback for checkpoint and early stopping
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="image_AUROC",
        save_last=True,
        verbose=True,
        auto_insert_metric_name=True,
        every_n_epochs=1,
    ),
    EarlyStopping(
        monitor="image_AUROC",
        mode="max",
        patience=10,
    ),
]

# 3 - instantiate the engine
engine = Engine(
    max_epochs=10,
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto",  # <"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=None,  # No logger
    task=TaskType.CLASSIFICATION,
)

# 4 - fit
print("Fit...")
engine.fit(datamodule=datamodule, model=model)

# 5 - test
print("Test...")
engine.test(datamodule=datamodule, model=model)

# 6 - export torch weights
print("Export weights...")
path_export_weights = engine.export(export_type=ExportType.TORCH, model=model)

print("path_export_weights: ", path_export_weights)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | EfficientAdModel         | 8.1 M  | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_metrics | MetricCollection         | 0      | train
3 | image_threshold       | F1AdaptiveThreshold      | 0      | train
4 | pixel_threshold       | F1AdaptiveThreshold      | 0 

Fit...


Training: |          | 0/? [00:00<?, ?it/s]

Calculate teacher channel mean & std: 100%|██████████| 60/60 [00:07<00:00,  7.57it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

...

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:03<00:00,  5.95it/s]
INFO: Epoch 9, global step 600: 'image_AUROC' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 9, global step 600: 'image_AUROC' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Test...


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            1.0            │
└───────────────────────────┴───────────────────────────┘

Export weights...
path_export_weights:  /content/results/EfficientAd/custom_dataset/latest/weights/torch/model.pt


In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision.transforms import Normalize

# Prepare device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Define normalization
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Initialize lists for labels and predictions
all_labels = []
all_predictions = []

# Test and gather predictions
with torch.no_grad():
    for i, data_test in enumerate(datamodule.test_dataloader()):
        images, labels = data_test["image"].to(device), data_test["label"].to(device)
        images = normalize(images)  # Normalize input images
        outputs = model(images)

        # Use 'map_ae' for predictions and apply thresholding
        threshold = 0.5
        predictions = (outputs["map_ae"] > threshold).float()  # Binary classification
        predictions = predictions.mean(dim=(1, 2, 3))  # Reduce spatial dimensions
        predictions = (predictions > threshold).long()  # Final binary class

        # Flatten labels if necessary
        labels = labels.view(-1)

        # Collect predictions and labels
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

# Calculate Accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_predictions, labels=[0, 1])
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=[0, 1])
disp.plot(cmap="viridis")
plt.title("Confusion Matrix")
plt.show()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Accuracy: 60.00%


In [10]:
import os
from PIL import Image
from anomalib.deploy import TorchInferencer
import numpy as np
import cv2
import torch
from torchvision.transforms.functional import to_tensor, normalize
import matplotlib.pyplot as plt

# Define paths
path_torch_model = "/content/results/EfficientAd/custom_dataset/latest/weights/torch/model.pt"  # Path to your saved Torch model
path_image = "/content/drive/MyDrive/praeciso/anomalib_dataset/test/anomalous/KO1_30K_97_03.bmp"  # Path to an existing image in your dataset
path_result = "/content/drive/MyDrive/praeciso/show"  # Directory to save results

# Ensure paths are valid
if not os.path.exists(path_torch_model):
    raise FileNotFoundError(f"Model file not found at {path_torch_model}")
if not os.path.exists(path_image):
    raise FileNotFoundError(f"Input image not found at {path_image}")
os.makedirs(path_result, exist_ok=True)

# Use GPU if available, otherwise fallback to CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1 - Instantiate the TorchInferencer
inferencer = TorchInferencer(path=path_torch_model, device=device)

# 2 - Load and preprocess the image
image = Image.open(path_image).convert("RGB")
image = image.resize((256, 256))  # Resize to match model input dimensions
image = to_tensor(image).float()  # Convert to tensor
image = normalize(image, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize

# 3 - Execute the prediction
result = inferencer.predict(image=image)

# 4 - Display the prediction result
if result.pred_label == 0:
    normal_score = 1 - result.pred_score
    print(f"Normal - pred_score: {normal_score:.4f}")
else:
    print(f"Abnormal - pred_score: {result.pred_score:.4f}")

# 5 - Build the bounding box from the mask
image_bbox = np.array(result.image.copy())
contours, _ = cv2.findContours(result.pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around detected anomalies
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image_bbox, (x, y), (x + w, y + h), (255, 0, 0), 2)

# Stack the mask three times to simulate the RGB color channels
mask = cv2.merge((result.pred_mask, result.pred_mask, result.pred_mask))

# 6 - Define a function to display and save images
def show_and_save_images(image_list, titles, save_path, grid=False):
    num_images = len(image_list)
    num_cols = 3
    num_rows = (num_images + num_cols - 1) // num_cols

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 10))
    axes = axes.flatten()

    for idx, (img, title) in enumerate(zip(image_list, titles)):
        axes[idx].imshow(img, cmap='gray' if len(img.shape) == 2 else None)
        axes[idx].set_title(title, fontsize=15)
        axes[idx].axis('off')

    # Turn off any remaining empty plots
    for ax in axes[num_images:]:
        ax.axis('off')

    # Save the result to the specified path
    save_file = os.path.join(save_path, "result.png")
    plt.savefig(save_file)
    print(f"Results saved at {save_file}")
    plt.show()

# 7 - Display and save the images
show_and_save_images(
    image_list=[
        result.image,
        result.heat_map,
        result.segmentations,
        mask,
        image_bbox
    ],
    titles=["Original Image", "Heat Map", "Segmentations", "Mask", "Image with Bounding Box"],
    save_path=path_result,
    grid=False
)


Abnormal - pred_score: 1.0000
Results saved at /content/drive/MyDrive/praeciso/show/result.png
